In [1]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path


In [2]:
print("cwd: " , os.getcwd())
HOME = str(Path.home())
# checkout from https://github.com/petermr/dictionary
# this is your local name
OPEN_DICT = os.path.join(HOME, "dictionary")

currentDictionaryTop = "openVirus202011"
# currentDictionaryTop = "openVirus202009"

OPEN_VIRUS_DICT = os.path.join(OPEN_DICT, currentDictionaryTop)
# dictionary folders
dictionaries = os.listdir(OPEN_VIRUS_DICT)
print("dictionaries " , dictionaries)


cwd:  /Users/pm286/dictionary
dictionaries  ['organization', 'test_trace', 'disease', 'zoonosis', 'npi', 'drug', 'country', 'virus']


In [3]:
def checkEntry(entry):
    """ check entry attributes"""
    attnames = set(['description', 'id', 'name', 'term', 'wikidataID', 'wikidataURL', 'wikipediaPage', 'wikipediaURL'])
    attribs = entry.attrib
    unknown_names = set()
    for attname in attribs:
#        print("att "+attname)
        if attname in attnames:
#            print("found "+attname)
            pass
        elif attname.startswith("_p"):
#            print("property "+attname)
            pass
        else:
            if not attname in unknown_names:
#                 print("unknown attname: ",attname)
                unknown_names.add(attname)
#                 print("un ", unknown_names)

    return unknown_names
            

In [4]:
def checkDesc(desc):
    """ add description"""
    pass

In [5]:
from lxml import etree
def checkFile(file, max_entry = 50):
    tree = ET.parse(file)
    root = tree.getroot()
    print("\nroot ", str(file), ": ", root.tag, " ~ ", str(root.attrib))
    count = 0
    unknown_count = 0
    max_unknown = 3
    for child in root[:max_entry]:
        if child.tag == "entry":
            unknown_names = checkEntry(child)
            if len(unknown_names) > 0 and unknown_count < max_unknown:
                unknown_count += 1
                print("unknown attributes ", unknown_names)
        elif child.tag == "desc":
            checkDesc(child)
        else: 
            print ("UNKNOWN TAG: ", child.tag)
        count += 1
    print("entries "+str(count))

    

In [6]:
def validateFile(file):
    
    # read openVirus schema
    xmlschema_file = os.path.join(OPEN_DICT, 'schema', 'openVirus_schema.xsd')
    xmlschema_doc = etree.parse(xmlschema_file)
    xmlschema = etree.XMLSchema(xmlschema_doc)
    
    doc = etree.parse(file)
#    doc = etree.XML(etree.tostring(root))
    validation_result = xmlschema.validate(doc)
    print(validation_result)

In [7]:
def edit(file, wikidataIdsToDelete):
    tree = ET.parse(file)
    root = tree.getroot()
    count = 0
    cleaned = []
    for child in root:
        if child.tag == "entry":
            wikidataId = child.attrib.get("wikidataId")
            if (wikidataId in wikidataIdsToDelete):
                print("deleted "+wikidataId)
                root.delete(child)
            else:
                cleaned.append(child)
        elif child.tag == "desc":
            cleaned.append(child)
    return cleaned
    print("entries "+str(count))

In [8]:
#file = os.path.join(OPEN_VIRUS, "bad.xml")
def validateDictionary(dictionary, dictionary_file, check=True, validate=True):
    if not os.path.exists(dictionary_file):
        print("dictionary file does not exist", dictionary_file)
        return False
    print("dictionary file: ", dictionary_file)
    if check:
        checkFile(dictionary_file)
    if validate:
        validateFile(dictionary_file)

In [9]:
def validateDictionaries(dictionary_root, check=True, validate=True):
    os.chdir(dictionary_root)
    if not os.path.exists(dictionary_root):
        print("dictionary root does not exist", dictionary_root)
        return False
    dictionary_names = os.listdir()
    for dictionary_name in dictionary_names:
        dictionary_folder = os.path.join(dictionary_root, dictionary_name)
        if not os.path.isdir(dictionary_folder):
            print("skipped", dictionary_folder)
            continue
        os.chdir(dictionary_folder)
        dictionary_file = os.path.join(dictionary_folder, dictionary_name + ".xml")
        validateDictionary(dictionary_name, dictionary_file, check, validate)


In [10]:
# validate single dictionary
check = True
validate = True
dictionary_name = "test_trace"
dictionary_file = os.path.join(OPEN_VIRUS_DICT, dictionary_name, dictionary_name+".xml")
validateDictionary(dictionary_name, dictionary_file, check, validate)

dictionary file:  /Users/pm286/dictionary/openVirus202011/test_trace/test_trace.xml

root  /Users/pm286/dictionary/openVirus202011/test_trace/test_trace.xml :  dictionary  ~  {'title': 'test_trace'}
entries 36
False


In [11]:
check = True
validate = True
validateDictionaries(OPEN_VIRUS_DICT, check, validate)

dictionary file:  /Users/pm286/dictionary/openVirus202011/organization/organization.xml

root  /Users/pm286/dictionary/openVirus202011/organization/organization.xml :  dictionary  ~  {'title': 'organization'}
unknown attributes  {'crossrefid', 'country'}
unknown attributes  {'crossrefid', 'country'}
unknown attributes  {'crossrefid', 'country'}
entries 50
False
dictionary file:  /Users/pm286/dictionary/openVirus202011/test_trace/test_trace.xml

root  /Users/pm286/dictionary/openVirus202011/test_trace/test_trace.xml :  dictionary  ~  {'title': 'test_trace'}
entries 36
False
dictionary file:  /Users/pm286/dictionary/openVirus202011/disease/disease.xml

root  /Users/pm286/dictionary/openVirus202011/disease/disease.xml :  dictionary  ~  {'title': 'disease'}
unknown attributes  {'Sanskrit', 'Spanish_description', 'Tamil', 'wikidataAltLabel', 'Kannada', 'Malayalam', 'Hindi', 'Tamil_wikipediaPage', 'German', 'French_description', 'German_altLabel', 'Chinese', 'German_description', 'Portuguese

In [12]:
"""   
deletedIds = set(["Q123", "Q234"])
file = os.path.join(OPEN_VIRUS, "country.xml")
file1 = os.path.join(OPEN_VIRUS, "bad_country.xml")
newroot= edit(file, deletedIds)
xml = ET.tostring(newroot, encoding="UTF-8")
print("X "+xml)
"""

'   \ndeletedIds = set(["Q123", "Q234"])\nfile = os.path.join(OPEN_VIRUS, "country.xml")\nfile1 = os.path.join(OPEN_VIRUS, "bad_country.xml")\nnewroot= edit(file, deletedIds)\nxml = ET.tostring(newroot, encoding="UTF-8")\nprint("X "+xml)\n'

In [14]:
import test_dict.py as test_dict


SyntaxError: invalid syntax (<ipython-input-14-ee59f793a27d>, line 1)